## imports and files

In [52]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import json
from copy import deepcopy
import random
import sys
import time
from typing import List

# get an absolute path to the directory that contains parent files
__file__ = globals()['_dh'][0]
project_dir = __file__ = globals()['_dh'][0]
sys.path.append(os.path.normpath(os.path.join(
    project_dir, '..', '..', '..')))

from experiments.utils.constants import (
    PIPLINES_PATH,
    NODE_PROFILING_CONFIGS_PATH,
    NODE_PROFILING_RESULTS_STATIC_PATH
)


In [2]:
series = 3
experiment_id = 2
config_key_mapper = "key_config_mapper.csv"

series_path = os.path.join(
    NODE_PROFILING_RESULTS_STATIC_PATH,
    'series',
    str(series))

## config map reader

In [3]:
def config_reader(csv_addr: str):
    config_path = os.path.join(
        series_path,
        csv_addr)
    key_config_df = pd.read_csv(config_path)
    return key_config_df
config_reader(config_key_mapper)

,experiment_id,pipeline_name,node_name,model_variant,cpu_request,memory_request,max_batch_size,max_batch_time,load,load_duration,series,series_meta,replicas
0,1,5-paper-video,resnet-human,resnet18,1,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
1,2,5-paper-video,resnet-human,resnet18,1,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
2,3,5-paper-video,resnet-human,resnet18,1,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
3,4,5-paper-video,resnet-human,resnet18,2,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
4,5,5-paper-video,resnet-human,resnet18,2,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,71,5-paper-video,resnet-human,resnet152,8,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
71,72,5-paper-video,resnet-human,resnet152,8,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
72,73,5-paper-video,resnet-human,resnet152,16,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
73,74,5-paper-video,resnet-human,resnet152,16,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1


In [4]:
def get_experiment_detail(experiment_id: int):
    path = os.path.join(
        series_path,        
        config_key_mapper
    )
    df = config_reader(config_key_mapper)
    return df[df["experiment_id"]==experiment_id]
get_experiment_detail(2)

,experiment_id,pipeline_name,node_name,model_variant,cpu_request,memory_request,max_batch_size,max_batch_time,load,load_duration,series,series_meta,replicas
1,2,5-paper-video,resnet-human,resnet18,1,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1


In [26]:
key_config_df = config_reader(config_key_mapper)
key_config_df

,experiment_id,pipeline_name,node_name,model_variant,cpu_request,memory_request,max_batch_size,max_batch_time,load,load_duration,series,series_meta,replicas
0,1,5-paper-video,resnet-human,resnet18,1,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
1,2,5-paper-video,resnet-human,resnet18,1,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
2,3,5-paper-video,resnet-human,resnet18,1,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
3,4,5-paper-video,resnet-human,resnet18,2,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
4,5,5-paper-video,resnet-human,resnet18,2,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,71,5-paper-video,resnet-human,resnet152,8,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
71,72,5-paper-video,resnet-human,resnet152,8,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
72,73,5-paper-video,resnet-human,resnet152,16,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
73,74,5-paper-video,resnet-human,resnet152,16,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1


In [6]:
def get_result_file_names(results_addr):
    files = []
    key_config = config_reader(config_key_mapper)
    experiments_keys = list(key_config["experiment_id"])
    for path in os.listdir(results_addr):
        if os.path.isfile(os.path.join(results_addr, path)):
            file_name = path.split(".")[0]
            try:
                if int(file_name) in experiments_keys:
                    files.append(path)
            except:
                pass
    return files
file_names = get_result_file_names(
    results_addr=series_path)
file_names

['38.json',
 '55.json',
 '69.json',
 '36.json',
 '6.json',
 '75.json',
 '52.json',
 '33.json',
 '11.json',
 '13.json',
 '9.json',
 '37.json',
 '45.json',
 '21.json',
 '50.json',
 '51.json',
 '54.json',
 '10.json',
 '25.json',
 '30.json',
 '29.json',
 '18.json',
 '62.json',
 '72.json',
 '61.json',
 '68.json',
 '56.json',
 '71.json',
 '66.json',
 '59.json',
 '70.json',
 '42.json',
 '4.json',
 '74.json',
 '63.json',
 '15.json',
 '20.json',
 '64.json',
 '73.json',
 '47.json',
 '67.json',
 '31.json',
 '8.json',
 '49.json',
 '1.json',
 '19.json',
 '3.json',
 '46.json',
 '28.json',
 '27.json',
 '22.json',
 '32.json',
 '23.json',
 '7.json',
 '17.json',
 '40.json',
 '5.json',
 '41.json',
 '60.json',
 '35.json',
 '2.json',
 '43.json',
 '14.json',
 '26.json',
 '24.json',
 '12.json',
 '57.json',
 '58.json',
 '44.json',
 '16.json',
 '39.json',
 '53.json',
 '48.json',
 '34.json',
 '65.json']

## Read resutls

In [34]:
def read_results(
    results_addr, return_type = "all", specific_file = None):
    files = get_result_file_names(results_addr=results_addr)
    
    if return_type == "all":
        return_results = {}
        for file in files:
            name = file.split(".")[0].split("/")[-1]
            full_path = os.path.join(
                results_addr, file
            )
            json_file = open(full_path)
            return_results[name] = json.load(json_file)
        return return_results

            
results = read_results(
    results_addr=series_path)
print(results.keys())


dict_keys(['38', '55', '69', '36', '6', '75', '52', '33', '11', '13', '9', '37', '45', '21', '50', '51', '54', '10', '25', '30', '29', '18', '62', '72', '61', '68', '56', '71', '66', '59', '70', '42', '4', '74', '63', '15', '20', '64', '73', '47', '67', '31', '8', '49', '1', '19', '3', '46', '28', '27', '22', '32', '23', '7', '17', '40', '5', '41', '60', '35', '2', '43', '14', '26', '24', '12', '57', '58', '44', '16', '39', '53', '48', '34', '65'])


In [8]:
def flatten_results(per_second_latencies):
    """
    change the results format from
    [[second_1], [second_2], ...]
    to:
    [req_1, req_2, ...]
    """
    flattend_results = []
    for second_results in per_second_latencies:
        for request_result in second_results:
            flattend_results.append(request_result)
    return flattend_results
flattened_results = flatten_results(results['1']['responses'])
len(flattened_results)

60

In [48]:
def latency_calculator(results):
    client_to_server_latencies = []
    model_latencies = []
    model_to_server_latencies = []
    latencies = {
        'client_to_server_latencies': [],
        'model_latencies': [],
        'model_to_server_latencies': [] 
    }
    timeout_count = 0
    for result in results:
        try:
            # outer times
            outter_times = result[
                'timing'] if 'timing' in result.keys() else result['time']
            sending_time = outter_times["sending_time"]
            arrival_time = outter_times["arrival_time"]
            # inner times
            data = result['outputs'][0]['data']
            data = json.loads(data[0])
            inner_times = data["time"]
            model_name = result['model_name']

            # TEMP to be fixed with a consistent time format
            inner_times = inner_times[model_name + '_times'][0]

            arrival_key = "arrival_" + model_name
            serve_key   = "serving_" + model_name
            model_arrival_time = inner_times[arrival_key]
            model_serving_time = inner_times[serve_key]
            # all three latencies
            client_to_server_latency =\
                model_arrival_time - sending_time
            model_latency =\
                model_serving_time - model_arrival_time
            model_to_server_latency =\
                arrival_time - model_serving_time
            client_to_server_latencies.append(client_to_server_latency)
            model_latencies.append(model_latency)
            model_to_server_latencies.append(model_to_server_latency)
            latencies = {
                'client_to_server_latencies': client_to_server_latencies,
                'model_latencies': model_latencies,
                'model_to_server_latencies': model_to_server_latencies
            }
        except KeyError:
            timeout_count += 1
    return latencies, timeout_count
experiment_id = 64
latencies, timeout_count = latency_calculator(
    flatten_results(results[str(experiment_id)]['responses']))
print(f'{timeout_count=}')
print(pd.DataFrame(latencies))

timeout_count=39
    client_to_server_latencies  model_latencies  model_to_server_latencies
0                     0.048265        22.886571                   0.315446
1                    21.901806        22.192502                   0.307734
2                    43.076366        15.699616                   0.204798
3                    58.203694        11.103679                   0.201899
4                    68.044231        17.897071                   0.201674
5                    85.309673         9.301476                   0.012162
6                    93.629958        10.101213                   0.104969
7                   103.639539         8.104300                   0.103414
8                   110.714560         9.205893                   0.200522
9                   119.976758        19.499093                   0.306861
10                  138.743904        15.699247                   0.402652
11                  153.242199        17.599520                   0.205070
12      

In [29]:
def metric_summary(metric, values):
    summary = {}
    if values != []:
        summary[f'{metric}_avg'] = np.average(values)
        summary[f'{metric}_p99'] = np.percentile(values, 99)
        summary[f'{metric}_p50'] = np.percentile(values, 50)
        summary[f'{metric}_var'] = np.var(values)
        summary[f'{metric}_max'] = max(values)
        summary[f'{metric}_min'] = min(values)
    else:
        summary[f'{metric}_avg'] = None
        summary[f'{metric}_p99'] = None
        summary[f'{metric}_p50'] = None
        summary[f'{metric}_var'] = None
        summary[f'{metric}_max'] = None
        summary[f'{metric}_min'] = None
    return summary

def latency_summary(latencies):
    summary = {}
    for metric_name, values in latencies.items():
        summary.update(
            metric_summary(
                metric=metric_name, values=values))
    return summary

summary = latency_summary(latencies=latencies)
pd.DataFrame(list(summary.items()))


,0,1
0,client_to_server_latencies_avg,0.031670
1,client_to_server_latencies_p99,0.173748
2,client_to_server_latencies_p50,0.021669
3,client_to_server_latencies_var,0.001198
4,client_to_server_latencies_max,0.219048
5,client_to_server_latencies_min,0.012388
6,model_latencies_avg,0.202863
7,model_latencies_p99,0.448301
8,model_latencies_p50,0.180912
9,model_latencies_var,0.005773


In [53]:
def result_processing():
    results = read_results(results_addr=series_path)            
    final_dataframe = []
    for experiment_id, result in results.items():
        processed_exp = {'experiment_id': experiment_id}
        latencies, timeout_count = latency_calculator(
            flatten_results(results[str(experiment_id)]['responses']))
        latencies = latency_summary(latencies)
        processed_exp.update(latencies)
        processed_exp['start_time'] = time.ctime(
            result['start_time_experiment'])
        processed_exp['end_time'] = time.ctime(
            result['end_time_experiment'])
        processed_exp['duration'] = round(
            result['end_time_experiment'] - result['start_time_experiment'])
        processed_exp['timeout_count'] = timeout_count
        skipped_metrics = [
            'time_cpu_usage_count',
            'time_cpu_usage_rate',
            'time_cpu_throttled_count',
            'time_cpu_throttled_rate',
            'time_memory_usage',
            'time_throughput',
            'responses',
            'start_time_experiment',
            'end_time_experiment'   
        ]
        for metric, values in result.items():
            if metric in skipped_metrics:
                continue
            processed_exp.update(metric_summary(metric=metric, values=values))
        final_dataframe.append(processed_exp)
    return pd.DataFrame(final_dataframe)
results = result_processing()
results

,experiment_id,client_to_server_latencies_avg,client_to_server_latencies_p99,client_to_server_latencies_p50,client_to_server_latencies_var,client_to_server_latencies_max,client_to_server_latencies_min,model_latencies_avg,model_latencies_p99,model_latencies_p50,...,memory_usage_p50,memory_usage_var,memory_usage_max,memory_usage_min,throughput_avg,throughput_p99,throughput_p50,throughput_var,throughput_max,throughput_min
0,38,6.247001,11.406209,8.448806,19.508946,11.412226,0.018592,1.057938,2.600192,0.888862,...,8.890081e+08,1.900194e+14,9.242911e+08,8.876605e+08,0.142227,0.485798,0.084034,0.025386,0.487395,0.000000
1,55,0.168033,1.563760,0.024060,0.114880,1.829748,0.018898,0.429202,1.317887,0.386089,...,1.238045e+09,7.384372e+14,1.296474e+09,1.237606e+09,0.252222,0.481499,0.249033,0.019112,0.490829,0.031400
2,69,28.936934,59.022370,29.368105,274.760224,61.181720,0.043805,1.958660,3.781435,1.807419,...,1.601114e+09,2.886490e+11,1.601917e+09,1.600451e+09,0.171149,0.489160,0.130253,0.029230,0.495798,0.000000
3,36,134.773946,270.109538,132.358389,7552.943862,273.716078,0.035219,5.576047,13.182573,4.150814,...,9.781985e+08,3.343036e+11,9.795584e+08,9.775636e+08,0.133403,0.205139,0.151261,0.003357,0.210084,0.000000
4,6,29.334845,55.145296,30.468241,237.262893,55.721272,0.036118,1.769544,3.339717,1.699381,...,6.881075e+08,4.426211e+13,6.940303e+08,6.741320e+08,0.234734,0.487395,0.231092,0.023303,0.495798,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,39,1.185270,2.515569,1.208970,0.472985,2.590326,0.020350,0.864255,1.469876,0.806272,...,8.882954e+08,3.786249e+14,9.298862e+08,8.882954e+08,0.120588,0.477395,0.004202,0.025000,0.487395,0.000000
71,53,16.971790,29.646670,18.569120,75.543646,29.953894,0.046224,1.450881,2.449106,1.308492,...,1.290170e+09,2.060659e+11,1.290473e+09,1.289388e+09,0.175840,0.485798,0.147059,0.026207,0.487395,0.000000
72,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.303253e+09,3.763651e+14,1.340776e+09,1.267159e+09,0.054435,0.100840,0.050420,0.000316,0.100840,0.033613
73,34,151.939223,292.619124,152.882025,8045.033599,293.482909,0.045008,7.713452,13.905752,7.404521,...,9.748070e+08,1.993844e+15,9.756836e+08,8.505344e+08,0.100529,0.134454,0.117647,0.001329,0.142857,0.000000


In [54]:
key_config_df = config_reader(config_key_mapper)
selected_key_config_df =  key_config_df[key_config_df['model_variant'] == 'resnet18']
selected_key_config_df

,experiment_id,pipeline_name,node_name,model_variant,cpu_request,memory_request,max_batch_size,max_batch_time,load,load_duration,series,series_meta,replicas
0,1,5-paper-video,resnet-human,resnet18,1,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
1,2,5-paper-video,resnet-human,resnet18,1,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
2,3,5-paper-video,resnet-human,resnet18,1,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
3,4,5-paper-video,resnet-human,resnet18,2,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
4,5,5-paper-video,resnet-human,resnet18,2,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
5,6,5-paper-video,resnet-human,resnet18,2,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
6,7,5-paper-video,resnet-human,resnet18,4,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
7,8,5-paper-video,resnet-human,resnet18,4,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
8,9,5-paper-video,resnet-human,resnet18,4,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
9,10,5-paper-video,resnet-human,resnet18,8,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1


In [39]:
def table_maker(selected_exp: List[int], selected_key_config_df):
    pass

    client_to_server_latencies  model_latencies  model_to_server_latencies
0                     0.035364         0.184394                   0.009770
1                     0.027726         0.066673                   0.009074
2                     0.020905         0.095008                   0.010575
3                     0.034425         0.042324                   0.008526
4                     0.019615         0.181757                   0.007569
5                     0.020600         0.063255                   0.008383
6                     0.021779         0.155235                   0.009538
7                     0.019918         0.066117                   0.008780
8                     0.022339         0.066038                   0.009091
9                     0.023826         0.173868                   0.008833
10                    0.018403         0.061095                   0.006266
11                    0.023032         0.054457                   0.009538
12                    0.0